# Homework 3
#### Team: Jair Aguirre and William Gottschalk, Friday Section

In [1]:
import pandas as pd
from collections import Counter
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from scipy.stats import zscore
%matplotlib inline

In [2]:
X = pd.read_table("trainingData.txt", header=None)
Y = pd.read_table("trainingTruth.txt", header=None)
print(X.shape)

Y = label_binarize(Y, classes=[1, 2, 3, 4])

(17378, 334)


In [4]:
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
filled_x = pd.DataFrame(imp.fit_transform(X))
std_norm_x = pd.DataFrame(zscore(filled_x))


X_train, X_test, y_train, y_test = train_test_split(
    std_norm_x.values, Y, test_size=0.5, random_state=1
)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(8689, 334) (8689, 334) (8689, 4) (8689, 4)


In [5]:
import time
start = time.time()

tuned_parameters = [
    {
        'estimator__kernel': ['rbf'],
        'estimator__gamma': [1e-2],
        'estimator__C': [1, 10, 20]
    }
]

clf_SVM = OneVsRestClassifier(SVC(), n_jobs=-1)
clf = GridSearchCV(clf_SVM, tuned_parameters, scoring='roc_auc')

clf.fit(X_train, y_train)

end = time.time()
print("training time (s): ", end - start)

training time (s):  850.8180170059204


In [6]:
 print("The best parameters are %s with a score of %0.2f"
      % (clf.best_params_, clf.best_score_))
pd.DataFrame(clf.cv_results_).sort_values("rank_test_score", axis=0)

The best parameters are {'estimator__kernel': 'rbf', 'estimator__gamma': 0.01, 'estimator__C': 10} with a score of 0.90


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_estimator__C,param_estimator__gamma,param_estimator__kernel,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,17.249081,22.409633,0.901327,1.0,10,0.01,rbf,"{'estimator__kernel': 'rbf', 'estimator__gamma...",1,0.897703,1.0,0.906251,1.0,0.900030,1.0,0.525846,0.318803,0.003608,0.0
2,17.034907,22.336555,0.901327,1.0,20,0.01,rbf,"{'estimator__kernel': 'rbf', 'estimator__gamma...",1,0.897703,1.0,0.906251,1.0,0.900030,1.0,0.385765,0.137554,0.003608,0.0
0,16.237610,21.789157,0.900206,1.0,1,0.01,rbf,"{'estimator__kernel': 'rbf', 'estimator__gamma...",3,0.896773,1.0,0.905183,1.0,0.898664,1.0,0.723448,0.309022,0.003603,0.0


In [7]:
model = OneVsRestClassifier(SVC(gamma=0.01, C=10))
model.fit(X_train, y_train)

print('done fitting...')

y_predict = model.predict(X_test)

print('done predicting...')

report = roc_auc_score(y_test, y_predict)
print("roc auc score is: ", report)

done fitting...
done predicting...
roc auc score is:  0.644653743497


In [20]:
# original report of running a basic SVM with no param tuning
print(report)

             precision    recall  f1-score   support

          1       0.83      0.92      0.87      3425
          2       0.66      0.69      0.67      2315
          3       0.78      0.93      0.84      3130
          4       0.57      0.16      0.25      1557

avg / total       0.74      0.76      0.73     10427

